Notebook to retrieve affordable housing data from CoStar (sample)

In [3]:
import pandas as pd 
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

In [42]:
pd.set_option('display.max_rows', None)

In [43]:
# Test / Demo

df = pd.read_clipboard(sep=r'^', skip_blank_lines=False)

In [45]:
df

,8800 Lombard Pl
0,Palisade
1,A
2,Existing
3,300
4,Hi-Rise
5,"289,041"
6,Apartments
7,San Diego
8,University City
9,San Diego
